In [63]:
import whisper
import torch
import pandas as pd
import os

# Check for cuda/cpu
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}')

# Load the model: large-v3-turbo or large-v3
model = whisper.load_model("large-v3-turbo", device=device)

Using cuda


Transcribe the given song, create list of "curses" words to search for

In [84]:
# I put a few tracks in here
song = 'juvenile'
audio_file = f"music/{song}.mp3"

# Format file path
notebook_dir = os.getcwd()
full_audio_path = os.path.join(notebook_dir, audio_file)

# word_timestamps=True for timestamp info
# Getting Triton kernel issues when I run this ?
result = model.transcribe(full_audio_path, word_timestamps=True)

c:\Users\dacla\anaconda3\Lib\site-packages\whisper\timing.py:42: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower median kernel implementation...
  warnings.warn(
c:\Users\dacla\anaconda3\Lib\site-packages\whisper\timing.py:146: UserWarning: Failed to launch Triton kernels, likely due to missing CUDA toolkit; falling back to a slower DTW implementation...
  warnings.warn(


In [99]:
# Edit for things to look for
curses = {'fuck', 'motherfucker', 'shit', 'bitch', 'nigga'}
to_add = []
all = []
# Collect words
for segment in result["segments"]:
    for word_info in segment['words']:
        all.append([word_info['word'].strip().lower(), float(word_info['start']), float(word_info['end']), word_info['probability']])
        for curse in curses:
            if curse in word_info['word'].strip().lower():
                to_add.append([word_info['word'].strip().lower(), float(word_info['start']), float(word_info['end']), word_info['probability']])
        
# This can probably be done better
# There needs to be some amount of text editing done to the automatic transcriptions:
# - Remove punctuation, make lowercase, separate root instances of common words 


# Create Dataframe
columns = ['word', 'start', 'end', 'prob']
df = pd.DataFrame(to_add, columns=columns)
df_all = pd.DataFrame(all, columns=columns)

In [101]:
# Inspect dataframe
df = df.drop_duplicates()
df_all = df_all.drop_duplicates()

df

,word,start,end,prob
0,shit,16.64,16.86,0.963379
1,nigga,19.82,20.02,0.825563
2,"motherfucker,",25.08,25.94,0.948235
4,fuck,28.38,28.56,0.725852
5,niggas,28.76,29.12,0.941863
6,bitches,32.24,32.50,0.976585
7,bitchin',33.90,34.48,0.412901
8,"bitch,",57.08,57.32,0.035671
9,nigga,58.68,59.08,0.894108
10,shit's,59.08,59.48,0.593813


In [83]:
# Save to csv
df.to_csv(f'music/{song}-curses.csv')

In [98]:
## Print full transcript
full_text = result["text"].strip()
print(full_text)

I'm charging 600 for some big ol's You do business with me, you coming back to get mo' My shit is fire cause it's jumping back to 29 So a nigga shouldn't have no problem, I'm be getting mad If you come to me with a short, I'ma say shop close Motherfucker, try to get smart, you gon' get popped, ho' I fuck when niggas like the Jack that will impact, ho' Had conversation for bitches until he got drool Bitchin' let you be gettin' twisted off the handle Baby, let Wamby, T and me come in and send, ho' Better beware because we bout to spend a bin, ho' Don't get caught up without the shiggas, you with end, ho' Me and my people gotta hustle just to pay to rent Some of us not in a position that we say we in Ain't have no milk collectors coming to my residence That's why every day I be runnin' for the president Bitch, all in I ain't another nigga shit's irrelevant You get your head busted for playin' with my intelligence My nanny's gonna die with me Pick up the supply with me Be up in the ride wi

This function does the actual editing

In [68]:
# Pydub does the audio effects
from pydub import AudioSegment

def reverse_audio_segment(input_audio_path, output_audio_path, times):
    """
    Reverses a specific segment of an audio track and saves the modified track.

    Args:
        input_audio_path (str): Path to the original audio file.
        output_audio_path (str): Path to save the modified audio file.
        times (list: int): Segments to reverse (in ms)
    """
    print(f'Applying edits to: {input_audio_path}')
    # Load the audio file
    audio = AudioSegment.from_file(input_audio_path)
    for (start_ms, end_ms) in times:
        
    # Ensure start and end times are within the audio duration
        if start_ms < 0 or end_ms > len(audio) or start_ms >= end_ms:
            print("Invalid start/end times. Please check the duration of the audio.")
            print(f"Audio duration: {len(audio)} ms")
            return

        # Select times to reverse
        before_segment = audio[:start_ms]

        target_segment = audio[start_ms:end_ms]
        reversed_segment = target_segment.reverse()

        after_segment = audio[end_ms:]

        # Concatenate
        audio = before_segment + reversed_segment + after_segment

    # Export the modified audio
    print(f'Outputting audio to {output_audio_path}')
    audio.export(output_audio_path, format="mp3") 
    return


Reverse the audio segments at the times identified by Whisper

In [102]:
# Path formatting
notebook_dir = os.getcwd()

input_file = f"music/{song}.mp3"
output_file = f"music/{song}-edit.mp3"

input_path = os.path.join(notebook_dir, input_file)
output_path = os.path.join(notebook_dir, output_file)

## Maybe a bit inefficient
times = []
for row in df.itertuples():
    word = row[1]
    start = int(row[2]*1000)
    end = int(row[3]*1000)
    times.append((start, end))

print(times)
# Run the reversing script
reverse_audio_segment(input_path, output_path, times)

[(16640, 16860), (19820, 20020), (25080, 25940), (28380, 28560), (28760, 29120), (32240, 32500), (33900, 34480), (57080, 57320), (58680, 59080), (59080, 59480), (93320, 93460), (98420, 98920), (104700, 104980), (121140, 121580), (126320, 126660), (140220, 140640), (172360, 172580), (192260, 192840), (193040, 193320), (250760, 251360), (254120, 254420), (255340, 256380)]
Applying edits to: c:\Users\dacla\Documents\auto-censoring\music/juvenile.mp3
Outputting audio to c:\Users\dacla\Documents\auto-censoring\music/juvenile-edit.mp3


Ideas: 
- demucs to split audio tracks

